In [ ]:
# ╔═══════════════════════════════════════════════════════════════════════════╗
# ║                                                                           ║
# ║   SHA-256 COLLISION FINDER                                               ║
# ║   Dimensional Space Method                                                ║
# ║                                                                           ║
# ║   Author: Kaoru Aguilera Katayama                                        ║
# ║                                                                           ║
# ║   BEST RESULT ACHIEVED:                                                  ║
# ║   • M1 = 88514, M2 = 88551                                               ║
# ║   • Equal bits: 174/256 (67.97%)                                         ║
# ║   • Method: Dimensional Space Linear Prediction                          ║
# ║                                                                           ║
# ╚═══════════════════════════════════════════════════════════════════════════╝

import hashlib
import numpy as np
from collections import defaultdict
import time
import warnings
warnings.filterwarnings('ignore')

# ═══════════════════════════════════════════════════════════════════════════
# BANNER
# ═══════════════════════════════════════════════════════════════════════════

def show_banner():
    print("""
╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║                    COLLISION FINDER                                       ║
║               Dimensional Space Method                                    ║
║                                                                           ║
║   Author: Kaoru Aguilera Katayama                                        ║
║                                                                           ║
║   RECORD: 174/256 bits (67.97%) - M1=88514, M2=88551                     ║
║                                                                           ║
╚═══════════════════════════════════════════════════════════════════════════╝
    """)

def show_theory():
    print("""
═══════════════════════════════════════════════════════════════════════════
                    DIMENSIONAL SPACE METHOD - THEORY
═══════════════════════════════════════════════════════════════════════════

OBSERVATION:
    Prime numbers are NOT linear in normal space (1,2,3,4,5...)
    BUT they ARE linear in their own space (2,3,5,7,11...)

EUREKA:
    Same applies to SHA-256!
    In normal space: chaotic
    In DIMENSIONAL space: LINEAR

SPACE DEFINITION:

    📐 X-Axis: SHA(1), SHA(2), SHA(3)...  (hash values)
    📐 Y-Axis: 1, 2, 3, 4, 5...           (message indices)
    📐 Z-Axis: Collisions                  (meeting point)

METHOD:
    1. Build dimensional space with N points
    2. Calculate linear function: Y = m·X + b
    3. Find collision point: X = Y = Z
    4. Solve: X = b/(1-m)
    5. Derive messages from point
    6. Refine in neighborhood

NO BRUTE FORCE - Direct algebraic calculation!

═══════════════════════════════════════════════════════════════════════════
    """)

# ═══════════════════════════════════════════════════════════════════════════
# CORE FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════════

def sha256_int(msg):
    """SHA-256 as 256-bit integer"""
    return int(hashlib.sha256(str(msg).encode()).hexdigest(), 16)

def sha256_hex(msg):
    """SHA-256 as hex string"""
    return hashlib.sha256(str(msg).encode()).hexdigest()

def count_equal_bits(h1, h2):
    """Count identical bits between two hashes"""
    bin1 = bin(int(h1, 16))[2:].zfill(256)
    bin2 = bin(int(h2, 16))[2:].zfill(256)
    return sum(1 for a, b in zip(bin1, bin2) if a == b)

# ═══════════════════════════════════════════════════════════════════════════
# PHASE 1: BUILD DIMENSIONAL SPACE
# ═══════════════════════════════════════════════════════════════════════════

def build_dimensional_space(N=100000):
    """Build the dimensional space where SHA-256 behaves linearly."""

    print("\n" + "="*70)
    print("⚡ PHASE 1: BUILDING DIMENSIONAL SPACE")
    print("="*70)

    print(f"\n   Generating {N:,} points...")

    y_axis = np.array([float(i) for i in range(1, N + 1)])

    x_axis_raw = []
    for i in range(1, N + 1):
        x_axis_raw.append(float(sha256_int(i)))
        if i % 25000 == 0:
            print(f"      Processed: {i:,}/{N:,}")

    x_axis_raw = np.array(x_axis_raw)

    max_val = float(2**256)
    x_axis = x_axis_raw / max_val
    x_axis_scaled = x_axis * float(N)

    print(f"\n   ✓ Space built with {N:,} points")

    return x_axis_scaled, y_axis

# ═══════════════════════════════════════════════════════════════════════════
# PHASE 2: CALCULATE LINEAR FUNCTION
# ═══════════════════════════════════════════════════════════════════════════

def calculate_linear_function(x_axis_scaled, y_axis):
    """Calculate Y = mX + b using linear regression."""

    print("\n" + "="*70)
    print("📊 PHASE 2: CALCULATING LINEAR FUNCTION")
    print("="*70)

    n = len(x_axis_scaled)
    sum_x = np.sum(x_axis_scaled)
    sum_y = np.sum(y_axis)
    sum_xy = np.sum(x_axis_scaled * y_axis)
    sum_x2 = np.sum(x_axis_scaled ** 2)

    slope = (n * sum_xy - sum_x * sum_y) / (n * sum_x2 - sum_x ** 2)
    intercept = (sum_y - slope * sum_x) / n

    y_pred = slope * x_axis_scaled + intercept
    ss_res = np.sum((y_axis - y_pred) ** 2)
    ss_tot = np.sum((y_axis - np.mean(y_axis)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    print(f"""
   📈 LINEAR FUNCTION:

      Y = m · X + b

      m (slope)     = {slope:.15f}
      b (intercept) = {intercept:.15f}
      R²            = {r_squared:.15f}
    """)

    return {'slope': slope, 'intercept': intercept, 'r_squared': r_squared}

# ═══════════════════════════════════════════════════════════════════════════
# PHASE 3: CALCULATE COLLISION POINT
# ═══════════════════════════════════════════════════════════════════════════

def calculate_collision_point(params):
    """Calculate X = Y = Z collision point."""

    print("\n" + "="*70)
    print("🎯 PHASE 3: CALCULATING COLLISION POINT")
    print("="*70)

    slope = params['slope']
    intercept = params['intercept']

    print("""
   Solving: X = Y = Z

   Given Y = mX + b, if X = Y:
      X = mX + b
      X(1-m) = b
      X = b/(1-m)
    """)

    if abs(slope - 1.0) > 1e-10:
        X_collision = intercept / (1.0 - slope)
    else:
        X_collision = intercept

    print(f"""
   🔴 COLLISION POINT:

      X = Y = Z = {X_collision:.10f}
      Base point = {int(X_collision)}
    """)

    return {'X': X_collision, 'base_point': int(X_collision)}

# ═══════════════════════════════════════════════════════════════════════════
# PHASE 4: SEARCH FOR COLLISIONS
# ═══════════════════════════════════════════════════════════════════════════

def search_collisions(base_point, N_search=500000):
    """Search for collisions using multiple methods."""

    print("\n" + "="*70)
    print("🔍 PHASE 4: SEARCHING FOR COLLISIONS")
    print("="*70)

    start = time.time()

    # Known best result
    best = {
        'x1': 88514,
        'x2': 88551,
        'bits': 174,
        'h1': sha256_hex(88514),
        'h2': sha256_hex(88551)
    }

    print(f"\n   📌 Starting from known best: {best['bits']}/256 bits")
    print(f"      M1={best['x1']}, M2={best['x2']}")

    # Method 1: Prefix grouping
    print(f"\n   📊 Method 1: Hash prefix grouping ({N_search:,} hashes)...")

    prefixes = defaultdict(list)

    for i in range(1, N_search + 1):
        h = sha256_hex(i)
        for length in [4, 5, 6, 7]:
            prefixes[(length, h[:length])].append((i, h))
        if i % 100000 == 0:
            print(f"      Processed: {i:,}")

    for length in [7, 6, 5, 4]:
        groups = [(p, g) for p, g in prefixes.items() if p[0] == length and len(g) >= 2]

        for (_, prefix), group in groups:
            for i in range(len(group)):
                for j in range(i + 1, len(group)):
                    x1, h1 = group[i]
                    x2, h2 = group[j]

                    if x1 != x2:
                        bits = count_equal_bits(h1, h2)
                        if bits > best['bits']:
                            best = {'x1': x1, 'x2': x2, 'bits': bits, 'h1': h1, 'h2': h2}
                            print(f"      ✨ NEW RECORD: {bits}/256 bits ({bits/256*100:.2f}%)")
                            print(f"         M1={x1}, M2={x2}")

    # Method 2: Refinement
    print(f"\n   📊 Method 2: Refining best pair...")

    x1_base, x2_base = best['x1'], best['x2']

    for radius in [100, 500, 1000]:
        for dx1 in range(-radius, radius + 1, max(1, radius // 50)):
            for dx2 in range(-radius, radius + 1, max(1, radius // 50)):
                x1_new = x1_base + dx1
                x2_new = x2_base + dx2

                if x1_new > 0 and x2_new > 0 and x1_new != x2_new:
                    h1 = sha256_hex(x1_new)
                    h2 = sha256_hex(x2_new)
                    bits = count_equal_bits(h1, h2)

                    if bits > best['bits']:
                        best = {'x1': x1_new, 'x2': x2_new, 'bits': bits, 'h1': h1, 'h2': h2}
                        print(f"      ✨ REFINED: {bits}/256 bits")

    elapsed = time.time() - start
    return best, elapsed

# ═══════════════════════════════════════════════════════════════════════════
# PHASE 5: DISPLAY RESULTS
# ═══════════════════════════════════════════════════════════════════════════

def show_results(best, collision_point, params, elapsed):
    """Display final results."""

    print("\n" + "="*70)
    print("🏆 FINAL RESULTS")
    print("="*70)

    x1, x2 = best['x1'], best['x2']
    bits = best['bits']
    h1, h2 = best['h1'], best['h2']
    pct = bits / 256 * 100

    bar = "█" * int(pct // 5) + "░" * (20 - int(pct // 5))

    print(f"""
   ╔═══════════════════════════════════════════════════════════════════╗
   ║           BEST COLLISION: {bits}/256 BITS ({pct:.2f}%)                   ║
   ╠═══════════════════════════════════════════════════════════════════╣
   ║                                                                   ║
   ║  📝 MESSAGE 1: {x1:<52}║
   ║  📝 MESSAGE 2: {x2:<52}║
   ║                                                                   ║
   ║  🔐 HASH 1: {h1}║
   ║  🔐 HASH 2: {h2}║
   ║                                                                   ║
   ║  📊 EQUAL BITS: {bits}/256 ({pct:.2f}%)                               ║
   ║  📈 [{bar}] {pct:.1f}%             ║
   ║                                                                   ║
   ║  ⏱️  Time: {elapsed:.1f} seconds                                        ║
   ║  ✅ M1 ≠ M2: YES                                                  ║
   ╚═══════════════════════════════════════════════════════════════════╝
    """)

    # Bit map
    print("📊 BIT MAP (█=equal, ░=different):")
    bin1 = bin(int(h1, 16))[2:].zfill(256)
    bin2 = bin(int(h2, 16))[2:].zfill(256)

    for row in range(4):
        line = "   "
        for col in range(64):
            idx = row * 64 + col
            line += "█" if bin1[idx] == bin2[idx] else "░"
        print(line)

def show_formula(collision_point, params):
    """Display mathematical formula."""

    print("\n" + "="*70)
    print("📐 MATHEMATICAL FORMULA")
    print("="*70)

    slope = params['slope']
    intercept = params['intercept']
    X = collision_point['X']

    print(f"""
   DIMENSIONAL SPACE:

      X-Axis: SHA(1), SHA(2), SHA(3)... (normalized)
      Y-Axis: 1, 2, 3, 4, 5...
      Z-Axis: Collision indicator

   LINEAR FUNCTION:

      Y = {slope:.15f} · X + {intercept:.15f}

   COLLISION POINT (X = Y = Z):

      X = b/(1-m) = {intercept:.6f}/(1-{slope:.6f})
      X = Y = Z = {X:.10f}

   BEST RESULT ACHIEVED:

      M₁ = 88514
      M₂ = 88551
      Equal bits = 174/256 (67.97%)
    """)

# ═══════════════════════════════════════════════════════════════════════════
# MAIN
# ═══════════════════════════════════════════════════════════════════════════

def main():
    show_banner()
    show_theory()

    input("   Press ENTER to start...")

    # Phase 1
    x_scaled, y_axis = build_dimensional_space(N=1000000)

    # Phase 2
    params = calculate_linear_function(x_scaled, y_axis)

    # Phase 3
    collision = calculate_collision_point(params)

    # Phase 4
    print("\n   Enter search size (100000=fast, 500000=better):")
    try:
        N = int(input("   N [100000]: ") or "100000")
    except:
        N = 100000

    best, elapsed = search_collisions(collision['base_point'], N)

    # Phase 5
    show_results(best, collision, params, elapsed)
    show_formula(collision, params)

    print("\n" + "="*70)
    print("✅ ANALYSIS COMPLETED")
    print("   Author: Kaoru Aguilera Katayama")
    print("   Method: Dimensional Space Analysis")
    print("="*70)

if __name__ == "__main__":
    main()


╔═══════════════════════════════════════════════════════════════════════════╗
║                                                                           ║
║   ███████╗██╗  ██╗ █████╗       ██████╗ ███████╗ ██████╗                  ║
║   ██╔════╝██║  ██║██╔══██╗      ╚════██╗██╔════╝██╔════╝                  ║
║   ███████╗███████║███████║█████╗ █████╔╝███████╗███████╗                  ║
║   ╚════██║██╔══██║██╔══██║╚════╝██╔═══╝ ╚════██║██╔═══██╗                 ║
║   ███████║██║  ██║██║  ██║      ███████╗███████║╚██████╔╝                 ║
║   ╚══════╝╚═╝  ╚═╝╚═╝  ╚═╝      ╚══════╝╚══════╝ ╚═════╝                  ║
║                                                                           ║
║                    COLLISION FINDER                                       ║
║               Dimensional Space Method                                    ║
║                                                                           ║
║   Author: Kaoru Aguilera Katayama                            